In [1]:
from blocks import FeatureExtraction
from models import *

In [2]:
import torch

In [3]:
input = torch.rand(3,3,64,46)
input.shape[1]

3

In [4]:
feature_extractor = FeatureExtraction(
    inc=input.shape[1],             # 输入通道数（如 RGB 图像）
    outc=3*input.shape[1],           # 输出通道数
    is_train=True,     # 可忽略（你代码中没使用）
    act='relu',        # 激活函数（仅在 need_RG=False 时有用）
    midc=[32, 32, 32], # 中间层通道数
    num_blocks=3,      # 块数
    need_RG=True,      # 是否使用 ResidualGroup 结构
    need_lwg=True      # 是否使用 LWGatedConv2D 做桥接
)


In [7]:
output = feature_extractor(input)
output.shape

torch.Size([3, 9, 64, 46])

In [8]:
r=1
hr_hidc = 2

In [9]:
gbufferConv=nn.Sequential(
            nn.Conv2d((10-3+1)*r,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            doubleResidualConv(64),
            nn.Conv2d(64,64,kernel_size=3,stride=1,padding=2,dilation=2),
            nn.ReLU(inplace=True),
            doubleResidualConv(64),
            nn.Conv2d(64,hr_hidc*2,kernel_size=3,stride=1,padding=2,dilation=2),
            nn.ReLU(inplace=True),
            doubleResidualConv(hr_hidc*2)
            )

In [10]:
input = torch.rand(32,8,64,64)

In [11]:
gbufferConv(input).shape

torch.Size([32, 4, 64, 64])

In [17]:
torch.randn(2,2) * 0.01

tensor([[-0.0095,  0.0119],
        [ 0.0231, -0.0022]])

In [11]:
int(5.1*64)

326

In [14]:
import torch
from models import DFSR  # 根据你的文件位置修改
import config                # 确保 config.Pixelshuffle 被设置

# 设置像素重排模式
config.Pixelshuffle = True   # or False
r = 5.1 if config.Pixelshuffle else 1

# 创建网络实例
model = DFSR(is_train=False)
model.eval()


# 模拟输入图像
N, H, W = 1, 32, 64  # 输入图像大小
high_h=int(H * r)
high_w=int(W * r) 
cur_lr = torch.randn(N, 3, H, W)                       # 假设是 RGB 图
cur_hr = torch.randn(N, 8, high_h+1, high_w)              # 假设是 [深度、法线、albedo、mask] 共 8 通道

# 前向推理
with torch.no_grad():
    sr_img = model(cur_lr, cur_hr,5.1)

# 输出尺寸
print("输出图像尺寸:", sr_img.shape)
# 预期为: [N, 3, H*r, W*r]，例如 [1, 3, 256, 256]（若 r=4）


输出图像尺寸: torch.Size([1, 1, 164, 326])


In [1]:
import torch
from calculate_flop import calculate_flop
from models import DFSR  # 修改为你的模型文件路径
import config

# 配置
config.Pixelshuffle = True
r = 4

# 初始化模型
model = DFSR(is_train=False)

# 创建 dummy 输入
N, H, W = 1, 64, 64
cur_lr = torch.randn(N, 3, H, W)
cur_hr = torch.randn(N, 8, H * r, W * r)
inputs = (cur_lr, cur_hr, r)

# 计算 FLOPs 和 参数量
flops_str, params_str = calculate_flop(model, inputs)

print("模型计算量 (FLOPs):", flops_str)
print("模型参数量 (Params):", params_str)


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
Raw FLOPs: 1.5228e+10, Raw Params: 1.0425e+06
Formatted FLOPs: 15.228G, Formatted Params: 1.043M
模型计算量 (FLOPs): 15.228G
模型参数量 (Params): 1.043M


In [5]:
import torch
from loss import CombinatorialLoss  # 修改为你的路径

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化损失函数
loss_fn = CombinatorialLoss(
    device=device,
    loss_weight=[1.0, 1.0, 1.0, 1.0],  # L1 + SSIM + perceptual + spatial mask
    use_gray_to_rgb=True
)

# 构造 dummy 输入
pred = torch.rand(1, 1, 64, 64).to(device)
label = torch.rand(1, 1, 64, 64).to(device)
mask = torch.ones(1, 1, 64, 64).to(device)

# 计算损失
loss_value = loss_fn(pred, label, mask)
print("组合损失值:", loss_value.item())


组合损失值: 209.88577270507812
